Interactive mode where you choose a word, and then multiple proposition for this word are given and you can choose to keep them or not with y/n.

In [1]:
import urllib
import os
from bs4 import BeautifulSoup
from IPython.core.display import HTML

In [2]:
import unicodedata

def strip_accents(s):
       return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

In [26]:
def crawl_word(word):
    page_url = "https://dico.elix-lsf.fr/dictionnaire/{}".format(word)
    page = urllib.request.urlopen(page_url)
    soup = BeautifulSoup(page)
    
    words = soup.find_all("section", {"class": "word"})
    #print(f"Found {len(words)} words")

    results = []
    for word in words:
        word_txt = word.find_all("h2", {"class": "word__title"})[0].text
        #print(f"# Word: {word_txt}")

        meanings = word.find_all("article", {"class": "meaning"})
        #print(f"\tFound {len(meanings)} meanings (taking only first one)")
        meaning = meanings[0]
        video = meaning.find_all("ul", {"class": "sign-list"})
        if len(video) == 0:
            continue
        video = video[0].findAll("video")
        if len(video) == 0:
            continue
        signs = meaning.find_all("ul", {"class": "sign-list"})
        #print(video[0]["src"], meaning.findAll("h3")[0].text)
        results.append({
            "word": word_txt, 
            "meaning": meaning.findAll("h3")[0].text, 
            "uri": video[0]["src"],
            "url": page_url
        })
    return results


def download_video(uri, path, folder="videos"):
    print(f"\t uri-> {uri}")
    video = urllib.request.urlopen(uri)
    tmp = video.read()
    with open(os.path.join(folder, path), "wb+") as f:
        f.write(tmp)
    

In [43]:
import re
from urllib.parse import quote

's%27il%20te%20pla%C3%AEt'

In [45]:
f = open("cards_sampled.csv", "a+")
   
word = "étudiant"
word = quote(word)
results = crawl_word(word)

for j in range(len(results)):
    norm_w = results[j]['word'].replace('(', '').replace(')', '').replace(' ', '_').replace('.', '')
    norm_w = strip_accents(norm_w)
    name = f"lsf_v2_{norm_w}_{j}.mp4"
    results[j]["video"] = f"[sound:{name}]"

    w = results[j]['word']
    m = results[j]['meaning']
    url = results[j]['url']
    uri = results[j]['uri']
    video = results[j]['video']

    print(w, m)
    y = input("Keep? y/n")
    if y == "y":
        print("Downloading...", end=" ")
        download_video(results[j]["uri"], name, "sampled_videos")
        #os.system(f"ffmpeg -i sampled_videos/{uri} -y -vcodec libx264 -crf 28 sampled_videos_opt/{uri};")
        f.write(f'"{w}";"{m}";{url};{uri};{video}\n')
        print("Done")
    else:
        print("Skipped")
        
f.close()

étudiant (adj.) 1. qui se rapporte aux étudiants.
Keep? y/nn
Skipped
étudiant (n.m.) 1. personne qui suit des études supérieures.
Keep? y/ny
Downloading... 	 uri-> https://www.elix-lsf.fr/IMG/mp4/etudiant_nm_1_1.mp4
Done
étudier (v.) 1. faire des études; se mettre à l'étude.
Keep? y/ny
Downloading... 	 uri-> https://www.elix-lsf.fr/IMG/mp4/etudier_v_1_1.mp4
Done
